In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.plotly as py
#py.set_credentials_file(username='raoshaheryarkhan', api_key='KswnKD2cSzUMp6zxf25p')
import plotly.figure_factory as ff
import math
from datetime import datetime
import sys
import os

In [2]:
import sys
project_path = "/home/aamna/Documents/apc/Amazon-Mining"
# make sure to use position 1
sys.path.insert(1, project_path)

In [3]:
from src.data.json_loader import JSONLoader

In [ ]:
# file_path corresponds to the file of the .gz file which contains the JSON file. 
file_path = '/home/aamna/Documents/apc/reviews_Electronics_5.json.gz'
metadata_path='/home/aamna/Documents/apc/meta_Electronics.json.gz'
loader = JSONLoader()

In [ ]:
data = loader.load_data(file_path)
#product=loader.load_data(metadata_path)

In [ ]:
data

In [ ]:
data.reviewTime = data.reviewTime .astype(str)
data['Year']=(data['reviewTime'].str[-4:])
data['Month']=(data['reviewTime'].str[0:2])
data['Day']=(data['reviewTime'].str[2:5])
data['Day']=data['Day'].str.replace(',','')
data['helpfulness_ratio']=(data['helpful'].str[0])/(data['helpful'].str[1])
data['review_length']=data['reviewText'].str.len()


In [ ]:
def word_count(row):
    str=row['reviewText']
    tokens=str.split()
    n_tokens=len(tokens)
    return n_tokens


In [ ]:
data['Wordcount'] = data.apply(lambda row :word_count(row), axis=1)

In [ ]:
def f(row):
    if row['overall'] == 3.0:
        val = 'Neutral'
    elif row['overall'] > 3.0:
        val = 'Positive'
    else:
        val = 'Negative'
    return val

In [ ]:
data['Reviewcategory'] = data.apply(f, axis=1)

In [ ]:
data['helpful']=data['helpful'].astype(str)
data['a'] = data['helpful'].apply(lambda x: x.split(',')[0])
data['b'] = data['helpful'].apply(lambda x: x.split(',')[1])
data['num']=data['a'].map(lambda x: x.lstrip('['))
data['deno']=data['b'].map(lambda x: x.rstrip(']'))
data['helpful_votes']=data['num'].astype(int)
data['total_votes']=data['deno'].astype(int)
data['unhelpful_votes']=data['total_votes'] - data['helpful_votes']
data['overall_helpfulness']=data['helpful_votes'] - data['unhelpful_votes']

In [ ]:
data

In [ ]:
data=data.drop('a', 1)
data=data.drop('b', 1)
data=data.drop('num', 1)
data=data.drop('deno', 1)

In [ ]:
data

In [ ]:


meanReview = data[['reviewerID','overall']].groupby(['reviewerID'],as_index=False).mean()
meanReview = meanReview.rename(columns = {'overall':'Avgrating'})



In [ ]:
reviewerCount = data[['reviewerID','asin']].groupby(['reviewerID'], as_index=False).count()
reviewerCount = reviewerCount.rename(columns = {'asin':'reviewCount'})
reviewer = pd.merge(reviewerCount, meanReview, on = "reviewerID")

In [ ]:
meanwordCount = data[['reviewerID','Wordcount']].groupby(['reviewerID'], as_index=False).mean()
meanwordCount = meanwordCount.rename(columns = {'Wordcount':'avg_wordcount'})
reviewer = pd.merge(reviewer, meanwordCount, on = "reviewerID")

In [ ]:
def to_YYYYMMDD(row):   
    datetime_object = datetime.strptime(row['reviewTime'], '%m %d, %Y').date()
    return datetime.strftime(datetime_object, "%Y%m%d")


In [ ]:


data['date(YYYYMMDD)'] = data.apply (lambda row: to_YYYYMMDD (row),axis=1)



In [ ]:
reviewerData = data[['reviewerID','reviewerName','date(YYYYMMDD)']].copy()
reviewerData 

In [ ]:
reviewerData = reviewerData.groupby('reviewerID').apply(lambda x: x.sort_values('date(YYYYMMDD)'))

In [ ]:


del reviewerData['reviewerID']
temp = reviewerData
temp = temp.groupby('reviewerID').head(1)



In [ ]:


temp = temp.reset_index()
del temp['level_1']



In [ ]:
reviewer_df = pd.merge(temp, reviewer, on = "reviewerID")

In [ ]:
sum_helpful_votes = data[['reviewerID','helpful_votes']].groupby(['reviewerID'],as_index=False).sum()
sum_helpful_votes = sum_helpful_votes .rename(columns = {'helpful_votes':'sum_helpful_votes'})

In [ ]:
sum_unhelpful_votes = data[['reviewerID','unhelpful_votes']].groupby(['reviewerID'],as_index=False).sum()
sum_unhelpful_votes = sum_unhelpful_votes .rename(columns = {'unhelpful_votes':'sum_unhelpful_votes'})

In [ ]:
reviewer_df = pd.merge(reviewer_df, sum_helpful_votes, on = "reviewerID")

In [ ]:
reviewer_df = pd.merge(reviewer_df, sum_unhelpful_votes, on = "reviewerID")

In [ ]:
reviewer_df['total_votes']=reviewer_df['sum_helpful_votes'] + reviewer_df['sum_unhelpful_votes']

In [ ]:
reviewer_df['overall_helpfulness']=reviewer_df['sum_helpful_votes'] - reviewer_df['sum_unhelpful_votes']

In [ ]:
reviewer_df

In [ ]:
reviewer_df.to_csv('reviwer_df.csv',index=False)

In [ ]:
meanReview_yearly = data.groupby(['reviewerID', 'Year'])['overall'].mean().reset_index()
meanReview_yearly 

In [ ]:
meanReview_yearly.to_csv('yearly.csv',index=False)